## LİNEER REGRESYON MODELİ İLE MASRAF TAHMİNİ

Veri Seti Hakkında Bilgi: ABD'nin bir yerleşim bölgesindeki sigortaya kayıtlı kişilerin;cinsiyet,sağlık sigortası kapsamındaki çocuk sayısı,yerleşim yeri ve benzeri bağımsız değişkenleriyle bağımlı değişken olan sigortaya kayıtlı kişinin faturalanan tıbbi masrafları verilmiştir.
Bu bilgilerle tıbbi masraf tahminlemesi yapılabilir mi onu test ettik.
Veri seti 1338 gözlem 7 parametreden oluşuyor.


* Gerekli kütüphanelerin import edilmesi ve  set_option ayarlarının yapılması

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 170)

* Verinin Okutulması

In [ ]:
df=pd.read_csv("../input/insurance/insurance.csv")

In [ ]:
df.head()

* Verinin boyut bilgisi

In [ ]:
df.shape

* Eksik değer kontrolü

In [ ]:
df.isnull().sum()

* Aykırı değer kontrolü

In [ ]:
#Aykırı değer yok gibi gözüküyor.
sns.boxplot(x=df["age"])
plt.show()

In [ ]:
#Çok fazla aykırı değer var.
sns.boxplot(x=df["charges"])
plt.show()

* Aykırı değerlerin baskılanması

In [ ]:
q1 = df["charges"].quantile(0.25)
q3 = df["charges"].quantile(0.75)
iqr = q3 - q1
up = q3 + 1.5 * iqr
low = q1 - 1.5 * iqr

In [ ]:
df.loc[df["charges"] < low,"charges"]=low

In [ ]:
df.loc[df["charges"] > up,"charges"]=up

In [ ]:
df.describe().T

In [ ]:
df.info()

* Kategorik, numerik columnlara erişilmesi için fonksiyon

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):    
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)
num_cols

* Veri setindeki numerik columnlar ile bağımlı değişken arasındaki korelasyona bakılması

In [ ]:
for col in num_cols:
    if col!="charges" :
        cor=df[[col, "charges"]].corr().loc[col, "charges"]
        print(col, cor)

In [ ]:
df.groupby("age")["charges"].mean()

* Columnları kategorilere ayırmak için bir fonksiyon

In [ ]:
def generate(df,c,a,b,t,y,y1,y2):
    df.loc[(df[c] < a), t] = y
    df.loc[(df[c] >= a) & (df[c] < b), t] = y1
    df.loc[(df[c] >= b), t] = y2
    return df[t]

In [ ]:
df["age"].describe()

* Age değişkeninin çeyrekliklerine denk gelen sayılara göre kategorilere ayırma işlemi

In [ ]:
generate(df,"age",27,51,"new_age","young","mature","senior")

In [ ]:
df.head()

*  Region parametresinde sınıf sayısı az olduğundan one hot encoding işleminde 1-0 olarak dönüşecek bu yüzden kategorilere ayırma işlemini yapmıyoruz.

In [ ]:
df["region"].unique()

In [ ]:
df["bmi"].describe()

* Vücut kitle indeksine göre bmi değerlerini kategorilere ayırma işlemi

In [ ]:
generate(df,"bmi",30,35,"bmi_risk","usual","risky","unhealthy")

* Çocuk sayısına göre kategorilere ayırma işlemi

In [ ]:
generate(df,"children",2,4,"children_C","A","B","C")

* İki farklı columnu birleştirmek işlemi için fonksiyon

In [ ]:
def twice_col(df,col1,col2):
    n=df[col1].unique().tolist()
    l=len(n)
    n1=df[col2].unique().tolist()
    l1=len(n1)
    for i in n:
        for j in range(l):
            for z in n1:
                for t in range(l1):
                    df.loc[(df[col1] == n[j]) & (df[col2] == n1[t]), str(col1) + "_" + str(col2)]=str(n[j]) +"_"+ str(n1[t])


* Çocuk sayısı ve sigara içip içmeme durumu da harcamaları etkileyeceğinden bu şekilde 6 ayrı sınıf oluşturulur

In [ ]:
twice_col(df,"children_C","smoker")

In [ ]:
df["children_C_smoker"].nunique()

In [ ]:
df.head()

* Çocuk sayısı verideki max. sayıda olduğunda ve sigara kullanılması durumunda ortalama harcama değeri

In [ ]:
liste=df["children_C_smoker"].unique().tolist()

In [ ]:
# 1 çocuk olduğunda veya hiç olmadığında harcamalar daha yüksekmiş.
for i in liste:
    a=df[df["children_C_smoker"]==i][["charges"]].mean()
    print(i,":",a,sep=" ")
    

In [ ]:
df[df["children_C_smoker"]=="C_yes"][["charges"]].mean()

* Çocuk sayısı ve cinsiyete göre harcamalara bakılması için bu şekilde bir column oluşturuyoruz.

In [ ]:
twice_col(df,"children_C","sex")

* Yaş ve cinsiyete göre bir column oluşturulması

In [ ]:
twice_col(df,"new_age","sex")

* Çocuk sayısı ve vücut kitle indeksine göre bir column oluşturulması

In [ ]:
twice_col(df,"children_C","bmi_risk")

In [ ]:
df.head()

In [ ]:
df.shape

* Veri setinin modele girebilmesi için encoding işlemlerinin yapılması

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float]
               and df[col].nunique() == 2]

In [ ]:
binary_cols 

In [ ]:
labelencoder = LabelEncoder()

In [ ]:
for i in binary_cols:
    df[i] = labelencoder.fit_transform(df[i])

In [ ]:
ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]

In [ ]:
df=pd.get_dummies(df, columns=ohe_cols, drop_first=True)

* Standartlaştırma işleminin yapılması

In [ ]:
for col in num_cols:
    transformer = MinMaxScaler().fit(df[[col]])
    df[col] = transformer.transform(df[[col]])

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

* Modelin kurulması

In [ ]:
from sklearn.linear_model import LinearRegression
y = df["charges"]
X = df.drop(["charges"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20, random_state=1)

reg_model = LinearRegression().fit(X, y)
reg_model.intercept_
reg_model.coef_

* Hata kareler ortalamasının karekökü olan rmse değeri 0'a ne kadar yakın olursa model mükemmele o kadar yakın olur.

In [ ]:
# Test RMSE
y_pred = reg_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

* Modelin veriyi açıklayabilme yüzdesi

In [ ]:
# Test RKARE
reg_model.score(X_test, y_test)

In [ ]:
np.mean(np.sqrt(-cross_val_score(reg_model,
                                 X,
                                 y,
                                 cv=10,
                                 scoring="neg_mean_squared_error")))
